Trial Data Bangkit Academy

In [2]:
#import dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DATASET/dataset_pemasukan_pengeluaran_2023.csv')
df.head(5)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
df.info()

In [ ]:
df.describe()

ttes